In [121]:
pip install opencv-python opencv-contrib-python argparse keras_facenet scipy scikit-learn numpy

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [122]:
from keras_facenet import FaceNet
embedder = FaceNet()


TRAINING

Data Collection
    - Faces Detection
    - Vector encodings of each face
    - labeling

In [123]:
import os,cv2,numpy as np

def CollectData(DIR,embedder=embedder):
    people = [name for name in os.listdir(DIR) if os.path.isdir(os.path.join(DIR,name))]

    features = [] # to store vector embedding of each face
    labels = [] # person's index in people array of each face detected

    image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif"} 

    for person in people :
        
        path = os.path.join(DIR,person)
        label = people.index(person)
        count = 0
        for img in os.listdir(path) :
            if not any(img.lower().endswith(ext) for ext in image_extensions): # running code only for image files
                continue
            
            img_path = os.path.join(path,img)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (800, 600))
            
            if img is None:
                continue
            
            detections = embedder.extract(img, threshold=0.95) # Every image should contain faces of only one person
            
            for detection in detections:
                features.append(detection['embedding'])
                labels.append(label)
                count += 1
            if count == 5 :
                break
            
            
    features = np.array(features,dtype=object)
    labels = np.array(labels)
    np.save('Memory/labels.npy',labels)
    np.save('Memory/features.npy',features)
    return features,labels,people

Data Preprocessing and Learning

In [124]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

def PreprocessAndLearn(features,labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    print("Dataset: train=%d, test=%d" % (np.shape(X_train)[0],(np.shape(X_test))[0])) 
    # normalize input vectors
    in_encoder = Normalizer()
    TrainX_norm = in_encoder.transform(X_train)
    TestX_norm = in_encoder.transform(X_test)
    # fit model
    model = SVC(kernel='linear', probability=True)
    model.fit(TrainX_norm, y_train)
    # predict
    yhat_train = model.predict(TrainX_norm)
    yhat_test = model.predict(TestX_norm)
    # score
    score_train = accuracy_score(y_train, yhat_train)
    score_test = accuracy_score(y_test, yhat_test)
    # summarize
    print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))
    return model, score_train,score_test

In [126]:
training_data='Faces/Sports_Persons'
features,labels,people = CollectData(training_data)
print(np.shape(features),np.shape(labels))
model,score_test,score_train=PreprocessAndLearn(features,labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
(25, 512) (25,)
Dataset: train=20, test=5
Accuracy

In [127]:
import joblib
trained_on = "Sports Celebraties"
metadata = {
    "model": model,
    "description": f"SVC model trained on {trained_on} classification",
    "target_names": people,
    "Accuracy of the model(train,test)":(score_train,score_test),
}
joblib.dump(metadata,f'Memory/Models/{trained_on}_trained_model.pkl')

['Memory/Models/Sports Celebraties_trained_model.pkl']

Prediction

In [128]:
import os,cv2,joblib
def predict_faces(img_path,model=None,people=None,trained_model=None):
    test_image = img_path
    if trained_model is not None :
        model_data = joblib.load(trained_model)
        people = model_data['target_names']
        model = model_data['model']
        print(model_data)
    in_encoder = Normalizer()
    print("Face Recognition among :",people)

    img = cv2.imread(test_image)
    if img is None:
        print('img not found')
        return
    img = cv2.resize(img, (800, 600))
    # cv2.imshow("test image",img)

    detections = embedder.extract(img, threshold=0.95)      # Face Detection
    if detections == [] :
        print("No Faces Detected")
        return
    embeddings = [each['embedding'] for each in detections] # Embedding collection
    TestX_norm = in_encoder.transform(embeddings)           # Normalizing  embedding
    labels = model.predict(TestX_norm)                      #Face recognition
    for i,detection in enumerate(detections):
        (x,y,w,h) = detection['box']
        label= labels[i]
        print(people[label])
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),thickness=2)
        cv2.putText(img,f"{people[label]}",(x-10,y-10),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,0),thickness=2)

        
    cv2.imshow("Detected faces",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Testing

In [129]:
# Test on above learned model (5 sports persons) : https://www.kaggle.com/datasets/vparh7/sports-celebrity-images
image_path = 'face_rec.png'   # Path to the image that needs to be predicted
predict_faces(image_path,model,people)



Face Recognition among : ['lionel_messi', 'maria_sharapova', 'roger_federer', 'serena_williams', 'virat_kohli']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
lionel_messi
virat_kohli
roger_federer


In [130]:
# To test on Pre Trained Top 100 indian Celebraties Data Set :  https://www.kaggle.com/datasets/sarthak1203/indian-celebrities?resource=download
image_path = 'image.png' 
model_path = f'Memory/Models/Indian_Celebraties_trained_model.pkl'
predict_faces(image_path,trained_model=model_path)


{'model': SVC(kernel='linear', probability=True), 'description': 'SVC model trained on Indian_Celebraties classification', 'target_names': ['Aamir_Khan', 'Abhay_Deol', 'Abhishek_Bachchan', 'Aftab_Shivdasani', 'Aishwarya_Rai', 'Ajay_Devgn', 'Akshaye_Khanna', 'Akshay_Kumar', 'Alia_Bhatt', 'Ameesha_Patel', 'Amitabh_Bachchan', 'Amrita_Rao', 'Amy_Jackson', 'Anil_Kapoor', 'Anushka_Sharma', 'Anushka_Shetty', 'Arjun_Kapoor', 'Arjun_Rampal', 'Arshad_Warsi', 'Asin', 'Ayushmann_Khurrana', 'Bhumi_Pednekar', 'Bipasha_Basu', 'Bobby_Deol', 'Deepika_Padukone', 'Disha_Patani', 'Emraan_Hashmi', 'Esha_Gupta', 'Farhan_Akhtar', 'Govinda', 'Hrithik_Roshan', 'Huma_Qureshi', 'Ileana', 'Irrfan_Khan', 'Jacqueline_Fernandez', 'John_Abraham', 'Juhi_Chawla', 'Kajal_Aggarwal', 'Kajol', 'Kangana_Ranaut', 'Kareena_Kapoor', 'Karisma_Kapoor', 'Kartik_Aaryan', 'Katrina_Kaif', 'Kiara_Advani', 'Kriti_Kharbanda', 'Kriti_Sanon', 'Kunal_Khemu', 'Lara_Dutta', 'Madhuri_Dixit', 'Manoj_Bajpayee', 'Mrunal_Thakur', 'Nana_Patekar',